In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# !git clone https://github.com/Pandas-Team/Autonomous-Vehicle-Environment-Perception.git
%cd /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception
!pip install -r requirements.txt

/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00


In [8]:
ls

Autonomous-Vehicle-Environment-Perception/  main.py           videos/
classes.txt                                 outputs/          weights/
elements/                                   PINet/            yolov5/
imgs/                                       README.md         yolov5s.pt
Instructions.ipynb                          requirements.txt
LICENSE                                     SGDepth/


In [9]:
# import unittest

# import torch
# import torch.nn as nn
# import torch.autograd as autograd


# class ScaleGrad(autograd.Function):
#     @staticmethod
#     def forward(ctx, scale, *inputs):
#         ctx.scale = scale

#         outputs = inputs[0] if (len(inputs) == 1) else inputs

#         return outputs

#     @staticmethod
#     def backward(ctx, *grad_outputs):
#         grad_inputs = tuple(
#             ctx.scale * grad
#             for grad in grad_outputs
#         )

#         return (None, *grad_inputs)


# class ScaledSplit(nn.Module):
#     """Identity maps an input into outputs and scale gradients in the backward pass

#     Args:
#         *grad_weights: one or multiple weights to apply to the gradients in
#             the backward pass

#     Examples:

#         >>> # Multiplex to two outputs, the gradients are scaled
#         >>> # by 0.3 and 0.7 respectively
#         >>> scp = ScaledSplit(0.3, 0.7)
#         >>> # The input may consist of multiple tensors
#         >>> inp
#         (tensor(...), tensor(...))
#         >>> otp1, otp2 = scp(inp)
#         >>> # Considering the forward pass both outputs behave just like inp.
#         >>> # In the backward pass the gradients will be scaled by the respective
#         >>> # weights
#         >>> otp1
#         (tensor(...), tensor(...))
#         >>> otp2
#         (tensor(...), tensor(...))
#     """

#     def __init__(self, *grad_weights):
#         super().__init__()
#         self.set_scales(*grad_weights)

#     def set_scales(self, *grad_weights):
#         self.grad_weights = grad_weights

#     def get_scales(self, *grad_weights):
#         return self.grad_weights

#     def forward(self, *inputs):
#         # Generate nested tuples, where the outer layer
#         # corresponds to the output & grad_weight pairs
#         # and the inner layer corresponds to the list of inputs
#         split = tuple(
#             tuple(ScaleGrad.apply(gw, inp) for inp in inputs)
#             for gw in self.grad_weights
#         )

#         # Users that passed only one input don't expect
#         # a nested tuple as output but rather a tuple of tensors,
#         # so unpack if there was only one input
#         unnest_inputs = tuple(
#             s[0] if (len(s) == 1) else s
#             for s in split
#         )

#         # Users that specified only one output weight
#         # do not expect a tuple of tensors put rather
#         # a single tensor, so unpack if there was only one weight
#         unnest_outputs = unnest_inputs[0] if (len(unnest_inputs) == 1) else unnest_inputs

#         return unnest_outputs


# class GRL(ScaledSplit):
#     """Identity maps an input and invert the gradient in the backward pass

#     This layer can be used in adversarial training to train an encoder
#     encoder network to become _worse_ a a specific task.
#     """

#     def __init__(self):
#         super().__init__(-1)


# class TestScaledSplit(unittest.TestCase):
#     def test_siso(self):
#         factor = 0.5

#         scp = ScaledSplit(factor)

#         # Construct a toy network with inputs and weights
#         inp = torch.tensor([ 1, 1, 1], dtype=torch.float32, requires_grad=True)
#         wgt = torch.tensor([-1, 0, 1], dtype=torch.float32, requires_grad=False)

#         pre_split = inp * wgt
#         post_split = scp.forward(pre_split)

#         self.assertTrue(torch.equal(pre_split, post_split), 'ScaledSplit produced non-identity in forward pass')

#         # The network's output is a single number
#         sum_pre = pre_split.sum()
#         sum_post = post_split.sum()

#         # Compute the gradients with and withou scaling
#         grad_pre, = autograd.grad(sum_pre, inp, retain_graph=True)
#         grad_post, = autograd.grad(sum_post, inp)

#         # Check if the scaling matches expectations
#         self.assertTrue(torch.equal(grad_pre * factor, grad_post), 'ScaledSplit produced inconsistent gradient')

#     def test_simo(self):
#         # TODO

#         pass

#     def test_miso(self):
#         # TODO

#         pass


#     def test_mimo(self):
#         # TODO

#         pass


# if __name__ == '__main__':
#     unittest.main(argv=['first-arg-is-ignored'], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.009s

OK


In [10]:
# import torch
# import torch.nn as nn

# from . import networks
# from . import layers


# class SGDepthCommon(nn.Module):
#     def __init__(self, num_layers, split_pos, grad_scales=(0.9, 0.1), pretrained=False):
#         super().__init__()

#         self.encoder = networks.ResnetEncoder(num_layers, pretrained)
#         self.num_layers = num_layers  # This information is needed in the train loop for the sequential training

#         # Number of channels for the skip connections and internal connections
#         # of the decoder network, ordered from input to output
#         self.shape_enc = tuple(reversed(self.encoder.num_ch_enc))
#         self.shape_dec = (256, 128, 64, 32, 16)

#         self.decoder = networks.PartialDecoder.gen_head(self.shape_dec, self.shape_enc, split_pos)
#         self.split = layers.ScaledSplit(*grad_scales)

#     def set_gradient_scales(self, depth, segmentation):
#         self.split.set_scales(depth, segmentation)

#     def get_gradient_scales(self):
#         return self.split.get_scales()

#     def forward(self, x):
#         # The encoder produces outputs in the order
#         # (highest res, second highest res, …, lowest res)
#         x = self.encoder(x)

#         # The decoder expects it's inputs in the order they are
#         # used. E.g. (lowest res, second lowest res, …, highest res)
#         x = tuple(reversed(x))

#         # Replace some elements in the x tuple by decoded
#         # tensors and leave others as-is
#         x = self.decoder(*x) # CHANGE ME BACK TO THIS

#         # Setup gradient scaling in the backward pass
#         x = self.split(*x)

#         # Experimental Idea: We want the decoder layer to be trained, so pass x to the decoder AFTER x was passed
#         # to self.split which scales all gradients to 0 (if grad_scales are 0)
#         # x = (self.decoder(*x[0]), ) + (self.decoder(*x[1]), ) + (self.decoder(*x[2]), )

#         return x

#     #def get_last_shared_layer(self):
#     #    return self.encoder.encoder.layer4


# class SGDepthDepth(nn.Module):
#     def __init__(self, common, resolutions=1):
#         super().__init__()

#         self.resolutions = resolutions

#         self.decoder = networks.PartialDecoder.gen_tail(common.decoder)
#         self.multires = networks.MultiResDepth(self.decoder.chs_x()[-resolutions:])

#     def forward(self, *x):
#         x = self.decoder(*x)
#         x = self.multires(*x[-self.resolutions:])
#         return x


# class SGDepthSeg(nn.Module):
#     def __init__(self, common):
#         super().__init__()

#         self.decoder = networks.PartialDecoder.gen_tail(common.decoder)
#         self.multires = networks.MultiResSegmentation(self.decoder.chs_x()[-1:])
#         self.nl = nn.Softmax2d()

#     def forward(self, *x):
#         x = self.decoder(*x)
#         x = self.multires(*x[-1:])
#         x_lin = x[-1]

#         return x_lin


# class SGDepthPose(nn.Module):
#     def __init__(self, num_layers, pretrained=False):
#         super().__init__()

#         self.encoder = networks.ResnetEncoder(
#             num_layers, pretrained, num_input_images=2
#         )

#         self.decoder = networks.PoseDecoder(self.encoder.num_ch_enc[-1])

#     def _transformation_from_axisangle(self, axisangle):
#         n, h, w = axisangle.shape[:3]

#         angles = axisangle.norm(dim=3)
#         axes = axisangle / (angles.unsqueeze(-1) + 1e-7)

#         # Implement the matrix from [1] with an additional identity fourth dimension
#         # [1]: https://en.wikipedia.org/wiki/Transformation_matrix#Rotation_2

#         angles_cos = angles.cos()
#         angles_sin = angles.sin()

#         res = torch.zeros(n, h, w, 4, 4, device=axisangle.device)
#         res[:,:,:,:3,:3] = (1 - angles_cos.view(n,h,w,1,1)) * (axes.unsqueeze(-2) * axes.unsqueeze(-1))

#         res[:,:,:,0,0] += angles_cos
#         res[:,:,:,1,1] += angles_cos
#         res[:,:,:,2,2] += angles_cos

#         sl = axes[:,:,:,0] * angles_sin
#         sm = axes[:,:,:,1] * angles_sin
#         sn = axes[:,:,:,2] * angles_sin

#         res[:,:,:,0,1] -= sn
#         res[:,:,:,1,0] += sn

#         res[:,:,:,1,2] -= sl
#         res[:,:,:,2,1] += sl

#         res[:,:,:,2,0] -= sm
#         res[:,:,:,0,2] += sm

#         res[:,:,:,3,3] = 1.0

#         return res

#     def _transformation_from_translation(self, translation):
#         n, h, w = translation.shape[:3]

#         # Implement the matrix from [1] with an additional dimension
#         # [1]: https://en.wikipedia.org/wiki/Transformation_matrix#Affine_transformations

#         res = torch.zeros(n, h, w, 4, 4, device=translation.device)
#         res[:,:,:,:3,3] = translation
#         res[:,:,:,0,0] = 1.0
#         res[:,:,:,1,1] = 1.0
#         res[:,:,:,2,2] = 1.0
#         res[:,:,:,3,3] = 1.0

#         return res

#     def forward(self, x, invert):
#         x = self.encoder(x)
#         x = x[-1]  # take only the feature map of the last layer ...

#         x_axisangle, x_translation = self.decoder(x)  # ... and pass it through the decoder

#         x_rotation = self._transformation_from_axisangle(x_axisangle)

#         if not invert:
#             x_translation = self._transformation_from_translation(x_translation)

#             return x_translation @ x_rotation

#         else:
#             x_rotation = x_rotation.transpose(3, 4)
#             x_translation = -x_translation

#             x_translation = self._transformation_from_translation(x_translation)

#             return x_rotation @ x_translation


# class SGDepth(nn.Module):
#     KEY_FRAME_CUR = ('color_aug', 0, 0)
#     KEY_FRAME_PREV = ('color_aug', -1, 0)
#     KEY_FRAME_NEXT = ('color_aug', 1, 0)

#     def __init__(self, split_pos=1, num_layers=18, grad_scale_depth=0.95, grad_scale_seg=0.05,
#                  weights_init='pretrained', resolutions_depth=1, num_layers_pose=18):

#         super().__init__()

#         # sgdepth allowed for five possible split positions.
#         # The PartialDecoder developed as part of sgdepth
#         # is a bit more flexible and allows splits to be
#         # placed in between sgdepths splits.
#         # As this class is meant to maximize compatibility
#         # with sgdepth the line below translates between
#         # the split position definitions.
#         split_pos = max((2 * split_pos) - 1, 0)

#         # The Depth and the Segmentation Network have a common (=shared)
#         # Encoder ("Feature Extractor")
#         self.common = SGDepthCommon(
#             num_layers, split_pos, (grad_scale_depth, grad_scale_seg),
#             weights_init == 'pretrained'
#         )

#         # While Depth and Seg Network have a shared Encoder,
#         # each one has it's own Decoder
#         self.depth = SGDepthDepth(self.common, resolutions_depth)
#         self.seg = SGDepthSeg(self.common)

#         # The Pose network has it's own Encoder ("Feature Extractor") and Decoder
#         self.pose = SGDepthPose(
#             num_layers_pose,
#             weights_init == 'pretrained'
#         )

#     def _batch_pack(self, group):
#         # Concatenate a list of tensors and remember how
#         # to tear them apart again

#         group = tuple(group)

#         dims = tuple(b.shape[0] for b in group)  # dims = (DEFAULT_DEPTH_BATCH_SIZE, DEFAULT_SEG_BATCH_SIZE)
#         group = torch.cat(group, dim=0)  # concatenate along the first axis, so along the batch axis

#         return dims, group

#     def _multi_batch_unpack(self, dims, *xs):
#         xs = tuple(
#             tuple(x.split(dims))
#             for x in xs
#         )

#         # xs, as of now, is indexed like this:
#         # xs[ENCODER_LAYER][DATASET_IDX], the lines below swap
#         # this around to xs[DATASET_IDX][ENCODER_LAYER], so that
#         # xs[DATASET_IDX] can be fed into the decoders.
#         xs = tuple(zip(*xs))

#         return xs

#     def _check_purposes(self, dataset, purpose):
#         # mytransforms.AddKeyValue is used in the loaders
#         # to give each image a tuple of 'purposes'.
#         # As of now these purposes can be 'depth' and 'segmentation'.
#         # The torch DataLoader collates these per-image purposes
#         # into list of them for each batch.
#         # Check all purposes in this collated list for the requested
#         # purpose (if you did not do anything wonky all purposes in a
#         # batch should be equal),

#         for purpose_field in dataset['purposes']:
#             if purpose_field[0] == purpose:
#                 return True

#     def set_gradient_scales(self, depth, segmentation):
#         self.common.set_gradient_scales(depth, segmentation)

#     def get_gradient_scales(self):
#         return self.common.get_gradient_scales()

#     def forward(self, batch):
#         # Stitch together all current input frames
#         # in the input group. So that batch normalization
#         # in the encoder is done over all datasets/domains.
#         dims, x = self._batch_pack(
#             dataset[self.KEY_FRAME_CUR]
#             for dataset in batch
#         )

#         # Feed the stitched-together input tensor through
#         # the common network part and generate two output
#         # tuples that look exactly the same in the forward
#         # pass, but scale gradients differently in the backward pass.
#         x_depth, x_seg = self.common(x)

#         # Cut the stitched-together tensors along the
#         # dataset boundaries so further processing can
#         # be performed on a per-dataset basis.
#         # x[DATASET_IDX][ENCODER_LAYER]
#         x_depth = self._multi_batch_unpack(dims, *x_depth)
#         x_seg = self._multi_batch_unpack(dims, *x_seg)

#         outputs = list(dict() for _ in batch)

#         # All the way back in the loaders each dataset is assigned one or more 'purposes'.
#         # For datasets with the 'depth' purpose set the outputs[DATASET_IDX] dict will be
#         # populated with depth outputs.
#         # Datasets with the 'segmentation' purpose are handled accordingly.
#         # If the pose outputs are populated is dependant upon the presence of
#         # ('color_aug', -1, 0)/('color_aug', 1, 0) keys in the Dataset.
#         for idx, dataset in enumerate(batch):
#             if self._check_purposes(dataset, 'depth'):
#                 x = x_depth[idx]
#                 x = self.depth(*x)
#                 x = reversed(x)

#                 for res, disp in enumerate(x):
#                     outputs[idx]['disp', res] = disp

#             if self._check_purposes(dataset, 'segmentation'):
#                 x = x_seg[idx]
#                 x = self.seg(*x)

#                 outputs[idx]['segmentation_logits', 0] = x

#             if self.KEY_FRAME_PREV in dataset:
#                 frame_prev = dataset[self.KEY_FRAME_PREV]
#                 frame_cur = dataset[self.KEY_FRAME_CUR]

#                 # Concatenating joins the previous and the current frame
#                 # tensors along the first axis/dimension,
#                 # which is the axis for the color channel
#                 frame_prev_cur = torch.cat((frame_prev, frame_cur), dim=1)

#                 outputs[idx]['cam_T_cam', 0, -1] = self.pose(frame_prev_cur, invert=True)

#             if self.KEY_FRAME_NEXT in dataset:
#                 frame_cur = dataset[self.KEY_FRAME_CUR]
#                 frame_next = dataset[self.KEY_FRAME_NEXT]

#                 frame_cur_next = torch.cat((frame_cur, frame_next), 1)
#                 outputs[idx]['cam_T_cam', 0, 1] = self.pose(frame_cur_next, invert=False)

#         return tuple(outputs)


ImportError: ignored

In [ ]:
# from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter


# class ArgumentsBase(object):
#     def __init__(self):
#         self.ap = ArgumentParser(
#             formatter_class=ArgumentDefaultsHelpFormatter
#         )

#     def running_args(self):
#         self.ap.add_argument('--weights-detector', type=str, default='weights/yolov5m.pt', help='weights for the main yolo detector')
#         self.ap.add_argument('--weights-sign'    , type=str, default='/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/SGDepth/weights/best_sign.pt', help='sign detector weights')
#         self.ap.add_argument('--weights-light'    , type=str, default='/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/SGDepth/weights/light_RegNety002.pth', help='sign detector weights')
#         self.ap.add_argument('--disp-detector', type=str, default='/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/SGDepth/weights/sgdepth_model.pth', help='disparity model weights')
#         self.ap.add_argument('--culane-model', type=str, default='/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/SGDepth/weights/culane_model.pkl', help='Culane model')
#         self.ap.add_argument('--video', type=str, default='videos/test4.mp4', help = 'The input video')
#         self.ap.add_argument('--save', action= 'store_true', help = 'Saving the output video')
#         self.ap.add_argument('--noshow', action= 'store_true', help =  'Do not Show the output frames')
#         self.ap.add_argument('--frame-drop', type = int, default = 1 , help =  'Frame Drop for processing')
#         self.ap.add_argument('--outputfps', type = int, default = 30 , help =  'Output Video FPS')
#         self.ap.add_argument('--fps', action = 'store_true' , help =  'Show fps')
#         self.ap.add_argument('--output-name', type = str ,default = 'output.mov' , help =  'Outputput video address')
#         self.ap.add_argument('--mode', type = str, default = 'day', help = 'Choose theprocessing model (day, night)')
#         self.ap.add_argument('--save-frames', action = 'store_true' , help =  'Saves individual Frames')


#     def SGDepth_harness_init_system(self):
#         self.ap.add_argument(
#             '--sys-cpu', default=False, action='store_true',
#             help='Disable Hardware acceleration'
#         )

#         self.ap.add_argument(
#             '--sys-num-workers', type=int, default=3,
#             help='Number of worker processes to spawn per DataLoader'
#         )

#         self.ap.add_argument(
#             '--sys-best-effort-determinism', default=False, action='store_true',
#             help='Try and make some parts of the training/validation deterministic'
#         )


#     def SGDepth_harness_init_model(self):

#         self.ap.add_argument(
#             '--model-num-layers', type=int, default=18, choices=(18, 34, 50, 101, 152),
#             help='Number of ResNet Layers in the depth and segmentation encoder'
#         )

#         self.ap.add_argument(
#             '--model-num-layers-pose', type=int, default=18, choices=(18, 34, 50, 101, 152),
#             help='Number of ResNet Layers in the pose encoder'
#         )

#         self.ap.add_argument(
#             '--model-split-pos', type=int, default=1, choices=(0, 1, 2, 3, 4),
#             help='Position in the decoder to split from common to separate depth/segmentation decoders'
#         )

#         self.ap.add_argument(
#             '--model-depth-min', type=float, default=0.3,
#             help='Depth Estimates are scaled according to this min/max',
#         )

#         self.ap.add_argument(
#             '--model-depth-max', type=float, default=80.0,
#             help='Depth Estimates are scaled according to this min/max',
#         )

#         self.ap.add_argument(
#             '--model-depth-resolutions', type=int, default=4, choices=(1, 2, 3, 4),
#             help='Number of depth resolutions to generate in the network'
#         )

#         self.ap.add_argument(
#             '--model-name', type=str, default='sgdepth_base',
#             help='A nickname for this model'
#         )

#         self.ap.add_argument(
#             '--model-load', type=str, default=None,
#             help='Load a model state from a state directory containing *.pth files'
#         )

#         self.ap.add_argument(
#             '--model-disable-lr-loading', default=False, action='store_true',
#             help='Do not load the learning rate scheduler if you load a checkpoint'
#         )


#     def _parse(self):
#         return self.ap.parse_args()


# class InferenceEvaluationArguments(ArgumentsBase):
#     def __init__(self):
#         super().__init__()
#         self.running_args()
#         self.SGDepth_harness_init_system()
#         self.SGDepth_harness_init_model()

#     def parse(self):
#         opt = self._parse()

#         return opt


In [ ]:
# # YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# """
# PyTorch Hub models https://pytorch.org/hub/ultralytics_yolov5/

# Usage:
#     import torch
#     model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
#     model = torch.hub.load('ultralytics/yolov5:master', 'custom', 'path/to/yolov5s.onnx')  # file from branch
# """

# import torch


# def _create(name, pretrained=True, channels=3, classes=80, autoshape=True, verbose=True, device=None):
#     """Creates or loads a YOLOv5 model

#     Arguments:
#         name (str): model name 'yolov5s' or path 'path/to/best.pt'
#         pretrained (bool): load pretrained weights into the model
#         channels (int): number of input channels
#         classes (int): number of model classes
#         autoshape (bool): apply YOLOv5 .autoshape() wrapper to model
#         verbose (bool): print all information to screen
#         device (str, torch.device, None): device to use for model parameters

#     Returns:
#         YOLOv5 model
#     """
#     from pathlib import Path

#     from models.common import AutoShape, DetectMultiBackend
#     from models.yolo import Model
#     from utils.downloads import attempt_download
#     from utils.general import LOGGER, check_requirements, intersect_dicts, logging
#     from utils.torch_utils import select_device

#     if not verbose:
#         LOGGER.setLevel(logging.WARNING)

#     check_requirements(exclude=('tensorboard', 'thop', 'opencv-python'))
#     name = Path(name)
#     path = name.with_suffix('.pt') if name.suffix == '' and not name.is_dir() else name  # checkpoint path
#     try:
#         device = select_device(('0' if torch.cuda.is_available() else 'cpu') if device is None else device)

#         if pretrained and channels == 3 and classes == 80:
#             model = DetectMultiBackend(path, device=device)  # download/load FP32 model
#             # model = models.experimental.attempt_load(path, map_location=device)  # download/load FP32 model
#         else:
#             cfg = list((Path(__file__).parent / 'models').rglob(f'{path.stem}.yaml'))[0]  # model.yaml path
#             model = Model(cfg, channels, classes)  # create model
#             if pretrained:
#                 ckpt = torch.load(attempt_download(path), map_location=device)  # load
#                 csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
#                 csd = intersect_dicts(csd, model.state_dict(), exclude=['anchors'])  # intersect
#                 model.load_state_dict(csd, strict=False)  # load
#                 if len(ckpt['model'].names) == classes:
#                     model.names = ckpt['model'].names  # set class names attribute
#         if autoshape:
#             model = AutoShape(model)  # for file/URI/PIL/cv2/np inputs and NMS
#         return model.to(device)

#     except Exception as e:
#         help_url = 'https://github.com/ultralytics/yolov5/issues/36'
#         s = f'{e}. Cache may be out of date, try `force_reload=True` or see {help_url} for help.'
#         raise Exception(s) from e


# def custom(path='path/to/model.pt', autoshape=True, _verbose=True, device=None):
#     # YOLOv5 custom or local model
#     return _create(path, autoshape=autoshape, verbose=_verbose, device=device)


# def yolov5n(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-nano model https://github.com/ultralytics/yolov5
#     return _create('yolov5n', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5s(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-small model https://github.com/ultralytics/yolov5
#     return _create('yolov5s', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5m(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-medium model https://github.com/ultralytics/yolov5
#     return _create('yolov5m', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5l(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-large model https://github.com/ultralytics/yolov5
#     return _create('yolov5l', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5x(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-xlarge model https://github.com/ultralytics/yolov5
#     return _create('yolov5x', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5n6(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-nano-P6 model https://github.com/ultralytics/yolov5
#     return _create('yolov5n6', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5s6(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-small-P6 model https://github.com/ultralytics/yolov5
#     return _create('yolov5s6', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5m6(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-medium-P6 model https://github.com/ultralytics/yolov5
#     return _create('yolov5m6', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5l6(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-large-P6 model https://github.com/ultralytics/yolov5
#     return _create('yolov5l6', pretrained, channels, classes, autoshape, _verbose, device)


# def yolov5x6(pretrained=True, channels=3, classes=80, autoshape=True, _verbose=True, device=None):
#     # YOLOv5-xlarge-P6 model https://github.com/ultralytics/yolov5
#     return _create('yolov5x6', pretrained, channels, classes, autoshape, _verbose, device)


# if __name__ == '__main__':
#     model = _create(name='yolov5s', pretrained=True, channels=3, classes=80, autoshape=True, verbose=True)
#     # model = custom(path='path/to/model.pt')  # custom

#     # Verify inference
#     from pathlib import Path

#     import numpy as np
#     from PIL import Image

#     from utils.general import cv2

#     imgs = [
#         'data/images/zidane.jpg',  # filename
#         Path('data/images/zidane.jpg'),  # Path
#         'https://ultralytics.com/images/zidane.jpg',  # URI
#         cv2.imread('data/images/bus.jpg')[:, :, ::-1],  # OpenCV
#         Image.open('data/images/bus.jpg'),  # PIL
#         np.zeros((320, 640, 3))]  # numpy

#     results = model(imgs, size=320)  # batched inference
#     results.print()
#     results.save()


In [ ]:
# from elements.yolo import YOLO, YOLO_Sign
# from elements.PINet import LaneDetection
# from elements.SGD import SGDepth_Model
# from elements.light_classifier import light_classifier
# from elements.asset import plot_object_colors, depth_estimator, apply_mask, apply_all_mask, ROI, plot_one_box, ui, horiz_lines
# import numpy as np
# import os
# import cv2
# from time import time as t
# import sys
# from datetime import timedelta
# from SGDepth.arguments import InferenceEvaluationArguments


# opt = InferenceEvaluationArguments().parse()


# if opt.noshow and not opt.save:
#     print("You're not getting any outputs!!\nExit")
#     sys.exit()

# # Load Models
# detector = YOLO()
# sign_detector = YOLO_Sign(opt.weights_sign)
# light_detector = light_classifier(opt.weights_light)
# lane_detector = LaneDetection(opt.culane_model)
# depth_seg_estimator = SGDepth_Model(opt.disp_detector)


# # Video Writer
# cap = cv2.VideoCapture("/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/SGDepth/videos/test7.mp4")
# frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
# h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# resize = not ((w == 1280) and (h == 720))


# if opt.save:
#     if len(opt.output_name.split('.'))==1:
#         opt.output_name += '.mp4'
#     output_video_folder = os.path.join('outputs/', opt.output_name.split('.')[0])
#     if opt.save_frames:
#         output_frames_folder = os.path.join(output_video_folder, 'frames')
#         os.makedirs(output_frames_folder, exist_ok=True)
#     output_video_name = os.path.join(output_video_folder, opt.output_name)
#     os.makedirs(output_video_folder, exist_ok = True)
#     print(output_video_folder)
#     w = cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 280
#     h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
#     out = cv2.VideoWriter(output_video_name,
#                             cv2.VideoWriter_fourcc(*'mp4v'),
#                             opt.outputfps, (int(w), int(h)))

# # Create color palettes for visualization
# obj_colors, sign_colors = plot_object_colors()

# frame_num = 0

# total_fps = []
# while(cap.isOpened()):

#     ret, frame = cap.read()
#     frame_num += 1
#     if not frame_num % opt.frame_drop == 0:
#         continue

#     if ret:
#         tc = t() # Start Time

#         if resize:
#             frame = cv2.resize(frame , (int(1280),int(720)))

#         main_frame = frame.copy()
#         yoloOutput = detector.detect(frame)
#         signOutput = sign_detector.detect_sign(frame)
#         depth, seg_img = depth_seg_estimator.inference(frame)


#         # # Dynamic ROI Generation
#         masked_image = ROI(main_frame, seg_img)


#         # ### Sidewalk detection ###
#         if opt.mode != 'night':
#             frame = apply_mask(frame, seg_img, mode = opt.mode)


#         # ### Lane Detection ###
#         frame = lane_detector.detect_lane(frame, masked_image)


#         # ### Object Detection ###
#         depth_values = []
#         for obj in yoloOutput.values:
#             xyxy = [int(obj[0]), int(obj[1]), int(obj[2]), int(obj[3])] # [Xmin, Ymin, Xmax, Ymax]

#             if obj[-2] in ['traffic light', 'stop sign', 'pedestrian'] :
#                 plot_one_box(xyxy, frame, label=obj[-2], color=obj_colors[obj[-2]], line_thickness=3)
#             else:
#                 ### Distance Measurement ###
#                 obj_area = (xyxy[3] - xyxy[1]) * (xyxy[2] - xyxy[0])
#                 if obj_area > 6000:
#                     depth_value = depth_estimator(xyxy, depth=depth, seg=seg_img, obj_name=obj[-1], mask_state=False)
#                     depth_values.append(depth_value)
#                     plot_one_box(xyxy, frame, distance=depth_value, label=obj[-2], color=obj_colors[obj[-2]], line_thickness=3)
#                 else:
#                     plot_one_box(xyxy, frame, label=obj[-2], color=obj_colors[obj[-2]], line_thickness=3)


#         ### Sign Detection ###
#         for sign in signOutput.values:
#             xyxy = [sign[0], sign[1], sign[2], sign[3]]
#             plot_one_box(xyxy, frame, label=sign[-1],  color=sign_colors[sign[-1]], line_thickness=3)

#         # ### Cross Walk Lines ###
#         # # frame = horiz_lines(main_frame, frame, mode = opt.mode)

#         ### UI ###
#         ui_bg = ui(main_frame, yoloOutput, light_detector, signOutput, depth_values)
#         frame = cv2.hconcat([frame, ui_bg])


#         t2 = t() # End of frame time
#         fps = (1/(t2-tc))
#         avg_fps = np.round(fps , 3)
#         estimated_time = (frame_count - frame_num) / avg_fps
#         estimated_time = str(timedelta(seconds=estimated_time)).split('.')[0]
#         s = "FPS : "+ str(fps)
#         if opt.fps:
#             cv2.putText(frame, s, (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), thickness= 2)


#         # Saving the output
#         if opt.save:
#             out.write(frame)
#             if opt.save_frames:
#                 cv2.imwrite(os.path.join(output_frames_folder , '{0:04d}.jpg'.format(int(frame_num))) , frame)


#         if not opt.noshow:
#             cv2.imshow('frame', frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#     else:
#         break

#     sys.stdout.write(
#         "\r[Input Video : %s] [%d/%d Frames Processed] [FPS : %f] [ET : %s]"
#         % (
#             opt.video,
#             frame_num,
#             frame_count,
#             fps,
#             estimated_time
#         )
#     )

# cap.release()

# if not opt.noshow:
#     cv2.destroyAllWindows()

In [ ]:
!cd /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception

## Download the weights and test videos and unzip the zip files

Both are anonymous links to download the weights and test videos.

In [11]:
!gdown 1X1uKaGENEBZamF6tOfx9eKLTIQLsBN5h
!gdown 1-bRFhDt5EZULnQaKO35U3oX-p6yZwteB

Downloading...
From: https://drive.google.com/uc?id=1X1uKaGENEBZamF6tOfx9eKLTIQLsBN5h
To: /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/weights.zip
100% 146M/146M [00:01<00:00, 78.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-bRFhDt5EZULnQaKO35U3oX-p6yZwteB
To: /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/videos.zip
100% 235M/235M [00:03<00:00, 71.6MB/s]


In [3]:
!unzip /content/drive/MyDrive/videos.zip
!unzip /content/drive/MyDrive/weights.zip

Archive:  /content/drive/MyDrive/videos.zip
replace videos/test1.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/weights.zip
replace weights/best_sign.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
cd /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/Autonomous-Vehicle-Environment-Perception

/content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/Autonomous-Vehicle-Environment-Perception


1) **Local:** video test[1-7].mp4
- **day:** videos test[1-5].mp4
- **night:** videos test[6, 7].mp4

2) **BDD100K:**
- **day:** videos test[8-10-11].mp4
- **night:** videos test[9].mp4

## Run!

- **--video**       [The input video]
- **--noshow**      [Do not Show the output frames]
- **--save**        [Saving the output video]
- **--output-name** [Outputput video name]
- **--mode** ['night', if you want to test the night test videos. default: 'day']

In [2]:
!pip install torch ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.9/626.9 kB 21.2 MB/s eta 0:00:00


In [3]:
!pip install timm

In [4]:
!python /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/Autonomous-Vehicle-Environment-Perception/main.py --video /content/drive/MyDrive/Autonomous-Vehicle-Environment-Perception/videos/test8.mp4 --noshow --save --output-name 'samveg_shah'

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: Ultralytics requirement /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 134.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.2s, installed 1 package: /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

Yolov5 model loaded!
requirements: /usr/local/lib/python3.10/dist-packages/requirements.txt not found, check failed.
YOLOv5 🚀 2023-7-8 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
newYOLOv5s summary: 213 layers, 7047883 parameters, 0 gradients
Adding AutoShape... 
Sign model loaded!
Traffic light classifier loaded!
CULane model loaded!
Downloading: "https://download.pytorch.org/model

The output video has been saved in **./outputs/myout/myout.mp4**